# Simple retrieval augmented generation with OpenAI

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
APPLY = True

import os

os.environ['SUPERDUPER_SECRETS_VOLUME'] = '~/data/secrets/snowflake_dev'

In [2]:
from superduper import superduper, CFG
import os

db = superduper('mongomock://', initialize_cluster=False)

2025-Jun-13 12:20:07.17| USER     | Duncans-MBP.fritz.box| build.py:87| Building Datalayer...
2025-Jun-13 12:20:07.25| USER     | Duncans-MBP.fritz.box| build.py:133| Building Datalayer... DONE


In [3]:
import json
import requests
import io
from superduper import logging


def getter():
    logging.info('Downloading data...')
    response = requests.get('https://superduperdb-public-demo.s3.amazonaws.com/text.json')
    logging.info('Downloading data... (Done)')
    data = json.loads(response.content.decode('utf-8'))
    return [{'x': r} for r in data]

In [4]:
if APPLY:
    data = getter()

In [5]:
from superduper import Table

table = Table('<var:table_name>', fields={'x': 'str'})

Create plugin:

In [6]:
from superduper import Plugin

plugin = Plugin(path='./rag_plugin.py')

<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [7]:
from superduper import Listener
from rag_plugin import Chunker

upstream_listener = Listener(
    model=Chunker(identifier='chunker'),
    select=db['<var:table_name>'],
    key='x',
    identifier='chunker',
    flatten=True,
    upstream=[plugin],
)

<!-- TABS -->
## Build text embedding model

OpenAI:

In [8]:
import os

from superduper_openai import OpenAIEmbedding

openai_embedding = OpenAIEmbedding(
    identifier='text-embedding',
    model='<var:embedding_model>',
    datatype='vector[float32:1536]',
)

## Create vector-index

In [9]:
from superduper import VectorIndex, Listener

vector_index_name = 'vectorindex'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs],
        model=openai_embedding,
        identifier='embeddinglistener',
        upstream=[upstream_listener],
    )
)

<!-- TABS -->
## Build LLM

In [10]:
from superduper_openai import OpenAIChatCompletion


llm_openai = OpenAIChatCompletion(
    identifier='llm-model',
    model='<var:llm_model>',
    datatype='str',
)

## Answer question with LLM

In [11]:
from rag_plugin import RAGModel


prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "These snippets are samples from our internal data-repositories, and should be used exclusively and as a matter"
    " of priority to answer the question. Please answer in 20 words or less.\n"
    "{context}\n"
    "Here is the question: {query}"
)


rag = RAGModel(
    'simple_rag',
    select=db[upstream_listener.outputs].select().like({upstream_listener.outputs: '<var:query>'}, vector_index=vector_index_name, n=5),
    prompt_template=prompt_template,
    key=upstream_listener.outputs,
    llm=llm_openai,
)

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [12]:
from superduper import Streamlit, Plugin
from rag_plugin import demo_func

demo = Streamlit('simple-rag-demo', demo_func=demo_func)

In [13]:
from superduper import Application

app = Application(
    'simple-rag-app',
    upstream=[table, plugin],
    components=[
        upstream_listener,
        vector_index,
        rag,
        demo,
    ],
    variables={
        'table_name': 'docs',
        'id_field': '_id',
        'embedding_model': 'text-embedding-ada-002',
        'llm_model': 'gpt-3.5-turbo',
    }
)

In [14]:
app.show()

Application:simple-rag-app:6c4f73b9d964d6faeb
├── upstream
│   ├── [0] Table:docs:3b21dc32e8beebd92b
│   │   ├── fields: {'x': 'str'}
│   │   └── primary_id: id
│   └── [1] Plugin:plugin-rag_plugin_py:b14248bd4605534489
│       ├── path: /Users/dodo/.superduper/plugins/5242564403d481dbe2/rag_plugin.py
│       └── cache_path: ~/.superduper/plugins
├── components
│   ├── [0] Listener:chunker:1862a8fc1629f85fe6
│   │   ├── upstream
│   │   │   └── [0] Plugin:plugin-rag_plugin_py:1dce9f8e8e852d4707
│   │   │       ├── path: /Users/dodo/.superduper/plugins/d74990b67d73e918c0/rag_plugin.py
│   │   │       └── cache_path: ~/.superduper/plugins
│   │   ├── cdc_table: docs
│   │   ├── key: x
│   │   ├── model: Chunker:chunker:b12cd565989b72fc4c
│   │   │   ├── datatype: str
│   │   │   └── chunk_size: 200
│   │   ├── select: docs
│   │   └── flatten: True
│   ├── [1] VectorIndex:vectorindex:851e610e9d2ace0b54
│   │   ├── cdc_table: _outputs__embeddinglistener__a67199bb7c0c1ac162
│   │   ├── indexing_listener: Listener:embeddinglistener:a67199bb7c0c1ac162
│   │   │   ├── upstream
│   │   │   │   └── [0] Listener:chunker:1862a8fc1629f85fe6
│   │   │   │       ├── upstream
│   │   │   │       │   └── [0] Plugin:plugin-rag_plugin_py:b2ce7e28d49fce8307
│   │   │   │       │       ├── path: /Users/dodo/.superduper/plugins/cad68136b064ea3d19/rag_plugin.py
│   │   │   │       │       └── cache_path: ~/.superduper/plugins
│   │   │   │       ├── cdc_table: docs
│   │   │   │       ├── key: x
│   │   │   │       ├── model: Chunker:chunker:b12cd565989b72fc4c
│   │   │   │       │   ├── datatype: str
│   │   │   │       │   └── chunk_size: 200
│   │   │   │       ├── select: docs
│   │   │   │       └── flatten: True
│   │   │   ├── cdc_table: _outputs__chunker__1862a8fc1629f85fe6
│   │   │   ├── key: _outputs__chunker__1862a8fc1629f85fe6
│   │   │   ├── model: OpenAIEmbedding:text-embedding:2b907268839faa545d
│   │   │   │   ├── datatype: vector
│   │   │   │   ├── model: text-embedding-ada-002
│   │   │   │   ├── max_batch_size: 8
│   │   │   │   └── batch_size: 100
│   │   │   └── select: _outputs__chunker__1862a8fc1629f85fe6
│   │   └── measure: cosine
│   ├── [2] RAGModel:simple_rag:233661fe822e51dd90
│   │   ├── datatype: str
│   │   ├── prompt_template: Use the following context snippets, these snippets are not ordered!, Answer the 
│   │   │   question based on this context.
│   │   │   These snippets are samples from our internal data-repositories, and should be used exclusively and as a
│   │   │   matter of priority to answer the question. Please answer in 20 words or less.
│   │   │   {context}
│   │   │   Here is the question: {query}
│   │   ├── select: _outputs__chunker__1862a8fc1629f85fe6.select().like({'_outputs__chunker__1862a8fc1629f85fe6': 
│   │   │   '<var:query>'}, "vectorindex", n=5)
│   │   ├── key: _outputs__chunker__1862a8fc1629f85fe6
│   │   └── llm: OpenAIChatCompletion:llm-model:ffce2ddfa3a6a5288f
│   │       ├── datatype: str
│   │       ├── model: gpt-3.5-turbo
│   │       ├── max_batch_size: 8
│   │       └── batch_size: 1
│   └── [3] Streamlit:simple-rag-demo:b33aaf6fa76d173336
│       └── demo_func: <function demo_func at 0x13c6de050>
└── variables: {'table_name': 'docs', 'id_field': '_id', 'embedding_model': 'text-embedding-ada-002', 'llm_model': 
    'gpt-3.5-turbo'}

In [15]:
if APPLY:
    db.apply(app, force=True)

2025-Jun-13 12:20:08.59| USER     | Duncans-MBP.fritz.box| apply.py:77| Found 14 create events to apply
2025-Jun-13 12:20:08.59| USER     | Duncans-MBP.fritz.box| apply.py:78| Found 17 jobs to apply
2025-Jun-13 12:20:08.59| USER     | Duncans-MBP.fritz.box| apply.py:93| Found these changes and/ or additions that need to be made:
2025-Jun-13 12:20:08.59| USER     | Duncans-MBP.fritz.box| apply.py:96| ----------------------------------------------------------------------------------------------------
2025-Jun-13 12:20:08.59| USER     | Duncans-MBP.fritz.box| apply.py:97| TABLE EVENTS:
2025-Jun-13 12:20:08.59| USER     | Duncans-MBP.fritz.box| apply.py:98| ----------------------------------------------------------------------------------------------------
2025-Jun-13 12:20:08.59| USER     | Duncans-MBP.fritz.box| apply.py:105| [0]: docs[x=str]
2025-Jun-13 12:20:08.59| USER     | Duncans-MBP.fritz.box| apply.py:105| [1]: _outputs__chunker__1862a8fc1629f85fe6[_outputs__chunker__1862a8fc1629

In [16]:
if APPLY:
    db['docs'].insert(data)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.34s/it]


['684bfb58177f8b02aaa513ec',
 '684bfb58177f8b02aaa513ed',
 '684bfb58177f8b02aaa513ee',
 '684bfb58177f8b02aaa513ef',
 '684bfb58177f8b02aaa513f0',
 '684bfb58177f8b02aaa513f1',
 '684bfb58177f8b02aaa513f2',
 '684bfb58177f8b02aaa513f3',
 '684bfb58177f8b02aaa513f4',
 '684bfb58177f8b02aaa513f5',
 '684bfb58177f8b02aaa513f6',
 '684bfb58177f8b02aaa513f7',
 '684bfb58177f8b02aaa513f8',
 '684bfb58177f8b02aaa513f9',
 '684bfb58177f8b02aaa513fa',
 '684bfb58177f8b02aaa513fb',
 '684bfb58177f8b02aaa513fc',
 '684bfb58177f8b02aaa513fd',
 '684bfb58177f8b02aaa513fe',
 '684bfb58177f8b02aaa513ff',
 '684bfb58177f8b02aaa51400',
 '684bfb58177f8b02aaa51401',
 '684bfb58177f8b02aaa51402',
 '684bfb58177f8b02aaa51403',
 '684bfb58177f8b02aaa51404',
 '684bfb58177f8b02aaa51405',
 '684bfb58177f8b02aaa51406',
 '684bfb58177f8b02aaa51407',
 '684bfb58177f8b02aaa51408',
 '684bfb58177f8b02aaa51409',
 '684bfb58177f8b02aaa5140a',
 '684bfb58177f8b02aaa5140b',
 '684bfb58177f8b02aaa5140c',
 '684bfb58177f8b02aaa5140d',
 '684bfb58177f

In [17]:
app.export('.', format='yaml')